# Pandas

In [6]:
import pandas as pd

df = pd.read_csv('./Data/ACS_16_5YR_B24011_with_ann.csv', usecols=['GEO.id', 'GEO.id2'], encoding='latin-1')
# df = df.drop(0)
df.head(2)

# excel sheet names
workbook = pd.ExcelFile('m1s3/Data/Yelp_Selected_Businesses.xlsx')
workbook.sheet_names

# save results
# df.to_csv('NewSavedView.csv', index=False) #Notice how we have to pass index=False if we do not want it included in our output

# supress scientific notation
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

# max rows / columns
# pd.options.display.max_rows = 99999
# pd.options.display.max_columns = 99999

FileNotFoundError: [Errno 2] No such file or directory: 'm1s3/Data/Yelp_Selected_Businesses.xlsx'

In [ ]:
# apply a function
df = pd.read_csv('m1s3/Data/turnstile_180901.txt')
def contains_n(text):
    return 'N' in text

df['On_N_Line'] = df.LINENAME.map(contains_n)
df.head()

In [ ]:
df = pd.read_csv('m1s3/Data/turnstile_180901.txt')

# change column data types
df.ENTRIES = df.ENTRIES.astype(int)
df.ENTRIES.dtype

# dates
# iloc slicing series / dataframes
print(df.DATE.iloc[0])
# pd.to_datetime(df.DATE, format='%m/%d/%Y').head()
pd.to_datetime(df.DATE).head()
# dt methods
df.DATE = pd.to_datetime(df.DATE)
df.DATE.dt.day_name().head()

# rename / drop columns, set new index
df = df.rename(columns = {'DATE': 'date'})
df.columns = [col.title().strip() for col in df.columns]
# df = df.drop('C/A', axis = 1) # If you don't pass the axis=1 parameter, pandas will try and drop a row with the specified index
# df = df.set_index('UNIT')
# df = df.reset_index()
df.head()

df.Date = pd.to_datetime(df.Date)
df['Dayofweek'] = df.Date.dt.dayofweek
grouped = df.groupby('Dayofweek').sum()
display(grouped.plot(kind = 'barh'))
grouped = grouped.reset_index()
grouped

grouped['IsWeekend'] = grouped.Dayofweek.map({0:False,1:False,2:False,3:False,4:False,5:True,6:True})
wkend = grouped.groupby('IsWeekend').mean()
display(wkend)
wkend[['Entries', 'Exits']].plot(kind='barh')

## Accessing Data

In [ ]:
# dataframes

df.head()
df.tail()
# df.info()
df.index
df.dtypes
df.columns
df.shape

df.iloc[5:80, 2:10] # integer-location based indexing
# df.loc[:, 'Linename'] # label-location based indexing
# or
df['Linename']
# boolean indexing
display(df.loc[(df['Entries'] > 7740888) & (df['Exits'] > 1367254), ['Station', 'Entries', 'Exits']][0:10])

# update values
df.loc[df['Entries'] > 7740888, ['Station']] = 'Station4'

# create new column
# df.loc[df['Entries'] > 7740888, 'NewCol'] = 'Busy'

In [ ]:
# series

linenames = df['Linename']
linenames.unique()
linenames.nunique()

## Statistical Methods

In [ ]:
df.describe()
df.mean()
df['Entries'].mean()
df['Entries'].quantile(.9) # get the value for 90% quantile for a specific column
df['Entries'].count()
df['Entries'].std()
df['Entries'].sum()
# pd.options.display.max_rows = 99999
# display(df['Entries'].cumsum()) # the cumulative sum, where each cell index contains the sum of all indices lower than, and including, itself
df['Station'].unique()
df['Station'].value_counts()

## .apply() / .applymap()

In [ ]:
# convert every value in the DataFrame to a string
string_df = df.applymap(lambda x: str(x))
string_df.info()

In [ ]:
df['Entries'].apply(lambda x: x+1)[0:1]

## Data Visualization with Pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook

plt.style.use('ggplot')

np.random.seed(777)

data = pd.DataFrame({'A':np.random.randn(365).cumsum(),
                    'B':np.random.randn(365).cumsum() + 25,
                    'C':np.random.randn(365).cumsum() - 25}, 
                     index = pd.date_range('1/1/2018', periods = 365))

# ; removes the '<matplotlib.axes._subplots.AxesSubplot at 0x8985400>' after the chart
data.plot('A', 'B', kind = 'SCATTER');

In [ ]:
# colormaps: https://matplotlib.org/examples/color/colormaps_reference.html
ax = data.plot.scatter('A', 'C', c = 'B', s = data['B'], colormap = 'viridis');
# setting the aspect ratio to equal allows the viewer to easily see that the range of series A is much smaller than series C
ax.set_aspect('equal')
ax.set_title('Manipulating Pandas plot objects in matplotlib')

In [ ]:
data.plot.box();
data.plot.hist(alpha = 0.7); # setting alpha level to inspect distribution overlap
data.plot.kde();

### Visualizing High Dimensional Data
#### Scatter Matrix / Parallel Plots

In [ ]:
iris = pd.read_csv('m1s3/Data/iris.csv')

# Scatter Matrix
pd.plotting.scatter_matrix(iris);

colormap = ('skyblue', 'salmon', 'lightgreen')
plt.figure()
# Parallel Plots
pd.plotting.parallel_coordinates(iris, 'species', color = colormap);

## Pivot Tables with Pandas

In [ ]:
df = pd.read_csv('m1s4/causes_of_death.tsv', sep='\t')
grouped = df.groupby(['State', 'Gender'])['Deaths', 'Population'].agg(['mean', 'min', 'max', 'std'])
grouped.head()

In [ ]:
grouped = grouped.reset_index()
cols0 = grouped.columns.get_level_values(0)
cols1 = grouped.columns.get_level_values(1)
grouped.columns = [col0 + '_' + col1 if col1 != '' else col0 for col0, col1 in list(zip(cols0, cols1))]
pivot = grouped.pivot(index='State', columns='Gender', values='Deaths_mean')
display(pivot.head())
pivot.plot(kind='barh', figsize=(15, 8))

# Data Cleansing
## Dealing with Missing Data
Strategies:
- Remove Data (Rows / Columns): **.dropna()**
- Replace
  - Continuous Data: Best to replace with the median: **.fillna()**
  - Categorical Data: Most common value?
- Keep
  - Continuous Data: **Coarse Classification (Binning)** Separate by categories (i.e. age range) and create a new category
  - Categorical Data: Replace with a string 'NaN' / 'missing'